<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 7:** 使用向量存储实现检索增强生成</font>

<br>

我们在前面的 notebook 中了解并尝试了嵌入模型。讨论了它在长文档比较中的应用，并以它为主干实现了基于语义的比较。本 notebook 将把这个思路用到检索模型上，探索如何靠*向量存储*来构建自动保存和检索信息的聊天机器人系统。

<br>

### **学习目标：**

* 理解语义相似度系统是怎么方便地实现检索的。
* 学会将检索模块整合到聊天模型系统中，以创建检索增强生成（RAG）工作流，用于完成文档检索或对话内存缓冲等任务。

<br>  

### **思考问题：**

* 本 notebook 不会尝试加入层次化推理（hierachical reasoning）或非朴素（non-naive）的 RAG，如规划智能体（palnning agents）。想想需要如何调整才能让这些组件在 LCEL 链中运行。
* 思考将向量存储方案用在规模化部署的最好时机是什么，以及什么时候需要用 GPU 进行优化。

<br>  

### **Notebook 版权声明：**

* 本 notebook 是 [**NVIDIA 深度学习培训中心**](https://www.nvidia.cn/training/)的课程[**《构建大语言模型 RAG 智能体》**](https://www.nvidia.cn/training/instructor-led-workshops/building-rag-agents-with-llms/)中的一部分，未经 NVIDIA 授权不得分发。

<br> 

### **环境设置：**

In [ ]:
# %%capture
## ^^ Comment out if you want to see the pip install process

## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

----

<br>

## 第 1 部分：RAG 工作流概述

此 notebook 将探索多个范式并给出参考代码，以帮助您开始使用最常见一些的检索增强工作流。具体来说将涵盖以下部分（每个部分各有侧重）：

<br>

> ***适用于交互式对话的向量存储工作流：***
* 为新对话生成语义嵌入。
* 将消息正文添加到向量存储以供检索。
* 在向量存储中查询相关消息填充到 LLM 上下文中。

<br>

> ***处理任意文档的工作流：***
* **将文档分快并处理成有用信息。**
* 为每个**新文档块**生成语义嵌入。
* 将**块正文（chunk bodies）**存到向量存储中以供检索。
* 在向量存储中查询相关的**块**，用来填充 LLM 上下文。
	+ ***可选：*修改/合成结果以获得更好的 LLM 结果。**

<br>

> **适用于任意文档目录的扩展工作流：**
* 将**每个文档**分为多个块并处理成有用的信息。
* 为每个新文档块生成语义嵌入。
* 将块正文存到**可扩展的向量数据库中以实现快速检索**。
	+ ***可选：*利用更大系统的层次化结构或元数据结构。**
* 在**向量数据库**中查询相关的块来填充 LLM 上下文。
	+ *可选：*修改/合成结果以获得更好的 LLM 结果。

<br>  

与 RAG 相关的一些重要术语都可以在 [**LlamaIndex Concepts 页面**](https://docs.llamaindex.ai/en/stable/getting_started/concepts.html) 查到，这是学习 LlamaIndex 加载和检索策略的很好的资源。我们强烈建议您在学习此 notebook 的过程中参考它，并鼓励您在课后试试 LlamaIndex 亲手体会它的优缺点！

> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/data_connection_langchain.jpeg" width=1200px/>
>
> From [**Retrieval | LangChain**🦜️🔗](https://python.langchain.com/docs/modules/data_connection/)

----

<br>  

## **第 2 部分：** 用于对话历史的 RAG

在之前的探索中，我们深入研究了文档嵌入模型的功能，并用它来嵌入、存储和比较文本的语义向量表示。尽管我们可以动手将其扩展到向量存储领域，但如果用标准 API 配合框架的话，就能发现它已经替我们完成了很多繁重的工作！

<br>

### **第 1 步：** 创建一段对话

想象一段 Llama-13B 聊天智能体和一只名为 Beras 的熊之间的对话。这段对话包含了大量细节和潜在的分支，为我们的研究提供了丰富的数据：

In [3]:
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]

仍然可以用上一个 notebook 的手动嵌入策略，但我们完全可以让向量数据库替我们做！

<br>

### **第 2 步：** 构建向量存储检索器

为了流程化对话中的相似性查询，我们可以使用向量存储来帮助我们追踪文本！**向量存储**（Vector Stores）或者叫向量存储系统，对嵌入/比较策略的大部分底层细节做了抽象，为加载和比较向量提供了一个简洁的接口。

> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/vector_stores.jpeg" width=1200px/>
>
> From [**Vector Stores | LangChain**🦜️🔗](https://python.langchain.com/docs/modules/data_connection/vectorstores/)

<br>

除了借助 API 简化流程外，向量存储还在背后实现了连接器（connector）、集成（integration）和优化。我们将从 [**FAISS 向量存储**](https://python.langchain.com/docs/integrations/vectorstores/faiss)开始，它集成了兼容 LangChain 的嵌入模型 [**FAISS (Facebook AI Similarity Search)**](https://github.com/facebookresearch/faiss)，从而允许在本地实现快速可扩展的流程！


**具体来说：**

1. 我们可以通过 `from_texts` 构造器将对话输入到 [**FAISS 向量存储**](https://python.langchain.com/docs/integrations/vectorstores/faiss)。这样我们的对话数据和嵌入模型就会用来创建索引。
2. 然后，这个向量存储就可以作为检索器，支持用 LangChain 运行时 API 来检索文档。

以下内容展示了如何构建 FAISS 向量存储并使用 LangChain `vectorstore` API 将其作为检索器使用：

In [4]:
%%time
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 59.5 ms, sys: 11.8 ms, total: 71.3 ms
Wall time: 810 ms


现在，检索器可以像任何其他可运行的 LangChain 一样用于查询向量存储中的某些相关文档：




In [5]:
pprint(retriever.invoke("What is your name?"))

[
    Document(
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    ),
    Document(
        page_content='[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for 
you![Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.'
    ),
    Document(
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    )
]

In [6]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    ),
    Document(
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]

如我们所见，检索工具从我们的查询中找到了一些语义相关的文档。您可能会注意到，不是所有文档都有用或清晰。比如，如果不是出于上下文，检索询问*“您的姓名”*时把*“Beras”*检索出来可能不是个好事。提前考虑到潜在的问题并让 LLM 组件相互协同更有可能让 RAG 达到好的效果。

<br>

### **第 3 步：** 将对话检索功能整合到我们的链中

现在，我们已把检索器组件作为一个链了，可以像以前一样将其整合到现有的聊天系统中。具体来说，我们现在可以构建一个***保持在线（always-on）的 RAG*** 了，其中：
* **默认情况下，检索器始终在检索上下文。**
* **生成器根据检索到的上下文执行操作。**

In [7]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [8]:
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    # | RPrint()
    | instruct_llm
    | StrOutputParser()
)

pprint(chain.invoke("Where does Beras live?"))

It seems that Beras lives in the Arctic. The cooler climate there is quite different from the warm climate found in
the Rocky Mountains, which Beras was interested in learning more about.

多试几个调用，看看新配置的效果。无论您选择的是哪个模型，都可以先从下面的几个问题开始。

In [9]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

Hello there! The Rocky Mountains are a stunning range of mountains that span across North America. Unfortunately, 
without more context, I can't give you a precise location beyond that. However, if you're interested, you can 
always look up more information online or watch documentaries about the Rocky Mountains to learn more!

In [10]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

Hello there! The Rocky Mountains are a stunning range of mountains that stretch across North America. To answer 
your question about their location, they do not directly border California. In fact, they run from Canada all the 
way to New Mexico, passing through the states of Montana, Idaho, Wyoming, Colorado, and a small part of northern 
New Mexico. I hope that helps clarify their location for you!

In [11]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

Hi there! Beras is a big blue bear and based on the context, we know that they live in the arctic. However, the 
context doesn't provide details about the exact location or the distance between the Rocky Mountains and the 
arctic. The Rocky Mountains are a prominent mountain range in North America, and the arctic region is also quite 
large. Therefore, it's not possible to provide an accurate distance without more specific information. I'm here to 
share that the Rocky Mountains are beautiful and majestic and stretch across North America, and I encourage Beras 
to learn more about them by doing some research online or watching documentaries.

<br>  

您可能会注意到把这个保持在线（always-on）的检索节点放到循环里效果很不错，因为目前输入 LLM 的上下文仍然相对较小。有必要反复尝试嵌入大小、上下文限制等配置，来更好地预测模型表现，并衡量为提高性能值得做出何种努力。

<br>

### **第 4 步：** 自动对话存储

现在向量存储已经可以工作了，我们最后再做一个集成：加一个调用 `add_texts` 更新存储状态的运行时。

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

########################################################################
## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

########################################################################

# instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

chat_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {input}"
    "\nAnswer the user conversationally. Make sure the conversation flows naturally.\n"
    "[Agent]"
)


conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | instruct_llm | StrOutputParser()})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, my favorite is honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

While I'm sure the Rocky Mountains would provide a magnificent backdrop for enjoying ice cream, it's worth noting 
that the Rocky Mountains are primarily known for their natural beauty and outdoor activities. Hiking, mountain 
biking, and wildlife viewing are popular activities. The Rocky Mountains are also home to a number of national 
parks, such as Banff National Park in Canada and Rocky Mountain National Park in the United States. These parks 
offer a range of activities for visitors of all ages and abilities. However, I'm always here to help you discover 
more about the world, and if you have any other questions about the Rocky Mountains or any other topic, please 
don't hesitate to ask!

While I can't truly guess your favorite food, Beras, from our conversation, it does seem that you are quite fond of
ice cream! It's always fun to enjoy a cool treat in a beautiful setting, like the Rocky Mountains.

Well Beras, I must admit, I made an assumption there! You're absolutely right that your love for ice cream was just
an observation and not a fact. Your fondness for honey is indeed delightful, just as the Rocky Mountains are! It's 
fascinating how diverse tastes can be, isn't it?

Well, based on our conversation, I can see that you indeed love ice cream! However, if I were to try and guess your
favorite food again, I would have to say honey, as you mentioned it being your actual favorite. But I'm an 
assistant, so I don't have the ability to know for certain unless you tell me. :)

不同于将上下文注入 LLM 的更自动化的全文本（full-text）或基于规则的方法，这样可避免上下文长度失控。这种策略虽然称不上完全可靠，但对于非结构化的对话来说已经是一个巨大的改进了（甚至不需要借助一个强大的指令微调模型做槽位填充）。

----

<br>

## **第 3 部分 [练习]：** 用 RAG 进行文档块检索

鉴于我们之前对文档加载的探索，您应该已经熟悉对数据块嵌入和检索了。现在值得花点时间继续过一遍，因为把 RAG 用在文档上是一把双刃剑：它看起来似乎开箱即用，但想让它在实际应用中保持可靠的性能需要非常谨慎地优化。我们也借此机会回顾一下基本的 LCEL 技能！

<br> 

### **练习：**

您可能还记得之前我们用 [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv) 加载了一些比较短的文章：

```python
from langchain.document_loaders import ArxivLoader

docs = [
    ArxivLoader(query="2205.00445").load(),  ## MRKL
    ArxivLoader(query="2210.03629").load(),  ## ReAct
]
```

根据所学，选择几个论文，并开发一个能讨论这些论文的聊天机器人！

<br>  

虽然这是一项相当艰巨的任务，但下面将提供**大部分**实现过程。演示过后，许多必须的环节就已经实现好了，您真正的任务是将它们集成到最终的 `retrieval_chain`。您会在最后一个 notebook 把它们集成到链中来完成评估测试！

<br>

### **任务 1：** 载入并分块您的文档

以下代码提供了一些可以载入到 RAG 链的默认论文。您可以根据需要选更多的论文，但要注意长文档的处理时间也更长。其中还有一些利于提高 RAG 性能的简化假设及处理步骤：

* 文档仅截取“参考“”（References）部分之前的内容。防止系统考虑冗长和不重要的引用和附录。
* 有一个能提供全局视角的列出所有可用文档的数据块。如果您的工作流并不是每次检索都提供元数据，那么这个数据块就会很有用，甚至可以在合适的时候作为更高优先级信息的一部分。
* 此外，还会插入元数据条目以提供常规信息。理想情况下，会有一些融合进了元数据的跨文档数据块。

**注意：** ***为执行评估，请至少放进一篇发表时间不超过一个月的论文！***

In [17]:
import json
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

## TODO: Please pick some papers and add them to the list as you'd like
## NOTE: To re-use for the final assessment, make sure at least one paper is < 1 month old
print("Loading Documents")
docs = [
    ArxivLoader(query="1706.03762").load(),  ## Attention Is All You Need Paper
    ArxivLoader(query="1810.04805").load(),  ## BERT Paper
    ArxivLoader(query="2005.11401").load(),  ## RAG Paper
    ArxivLoader(query="2205.00445").load(),  ## MRKL Paper
    ArxivLoader(query="2310.06825").load(),  ## Mistral Paper
    ArxivLoader(query="2306.05685").load(),  ## LLM-as-a-Judge
    ArxivLoader(query="2502.09891").load(),
    ArxivLoader(query="2502.09304").load(),
    ## Some longer papers
    ArxivLoader(query="2210.03629").load(),  ## ReAct Paper
    ArxivLoader(query="2112.10752").load(),  ## Latent Stable Diffusion Paper
    ArxivLoader(query="2103.00020").load(),  ## CLIP Paper
    ## TODO: Feel free to add more
]

## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = json.dumps(doc[0].page_content)
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title')
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - # Chunks: {len(chunks)}")
    print(f" - Metadata: ")
    pprint(chunks[0].metadata)
    print()

Loading Documents
Chunking Documents


Available Documents:
 - Attention Is All You Need
 - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
 - Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
 - MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external knowledge 
sources and discrete reasoning
 - Mistral 7B
 - Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena
 - ArchRAG: Attributed Community-based Hierarchical Retrieval-Augmented Generation
 - KET-RAG: A Cost-Efficient Multi-Granular Indexing Framework for Graph-RAG
 - ReAct: Synergizing Reasoning and Acting in Language Models
 - High-Resolution Image Synthesis with Latent Diffusion Models
 - Learning Transferable Visual Models From Natural Language Supervision 

Document 0
 - # Chunks: 35
 - Metadata: 


{
    'Published': '2023-08-02',
    'Title': 'Attention Is All You Need',
    'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz 
Kaiser, Illia Polosukhin',
    'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural 
networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder 
through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on 
attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation
tasks show these models to be\nsuperior in quality while being more parallelizable and requiring 
significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation 
task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 
English-to-French\ntranslation task, our model establishes a new single-model state-of-the-art\nBLEU score of 41.8 
after training for 3.5 days on eight GPUs, a small fraction\nof the training costs of the best models from the 
literature. We show that the\nTransformer generalizes well to other tasks by applying it successfully to\nEnglish 
constituency parsing both with large and limited training data.'
}


Document 1
 - # Chunks: 45
 - Metadata: 


...

{
    'Published': '2023-03-10',
    'Title': 'ReAct: Synergizing Reasoning and Acting in Language Models',
    'Authors': 'Shunyu Yao, Jeffrey Zhao, Dian Yu, Nan Du, Izhak Shafran, Karthik Narasimhan, Yuan Cao',
    'Summary': 'While large language models (LLMs) have demonstrated impressive capabilities\nacross tasks in 
language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought 
prompting) and acting (e.g.\naction plan generation) have primarily been studied as separate topics. In 
this\npaper, we explore the use of LLMs to generate both reasoning traces and\ntask-specific actions in an 
interleaved manner, allowing for greater synergy\nbetween the two: reasoning traces help the model induce, track, 
and update\naction plans as well as handle exceptions, while actions allow it to interface\nwith external sources, 
such as knowledge bases or environments, to gather\nadditional information. We apply our approach, named ReAct, to 
a diverse set of\nlanguage and decision making tasks and demonstrate its effectiveness over\nstate-of-the-art 
baselines, as well as improved human interpretability and\ntrustworthiness over methods without reasoning or acting
components.\nConcretely, on question answering (HotpotQA) and fact verification (Fever),\nReAct overcomes issues of
hallucination and error propagation prevalent in\nchain-of-thought reasoning by interacting with a simple Wikipedia
API, and\ngenerates human-like task-solving trajectories that are more interpretable than\nbaselines without 
reasoning traces. On two interactive decision making\nbenchmarks (ALFWorld and WebShop), ReAct outperforms 
imitation and\nreinforcement learning methods by an absolute success rate of 34% and 10%\nrespectively, while being
prompted with only one or two in-context examples.\nProject site with code: https://react-lm.github.io'
}


Document 9
 - # Chunks: 52
 - Metadata: 


{
    'Published': '2022-04-13',
    'Title': 'High-Resolution Image Synthesis with Latent Diffusion Models',
    'Authors': 'Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer',
    'Summary': 'By decomposing the image formation process into a sequential application of\ndenoising 
autoencoders, diffusion models (DMs) achieve state-of-the-art\nsynthesis results on image data and beyond. 
Additionally, their formulation\nallows for a guiding mechanism to control the image generation process 
without\nretraining. However, since these models typically operate directly in pixel\nspace, optimization of 
powerful DMs often consumes hundreds of GPU days and\ninference is expensive due to sequential evaluations. To 
enable DM training on\nlimited computational resources while retaining their quality and flexibility,\nwe apply 
them in the latent space of powerful pretrained autoencoders. In\ncontrast to previous work, training diffusion 
models on such a representation\nallows for the first time to reach a near-optimal point between 
complexity\nreduction and detail preservation, greatly boosting visual fidelity. By\nintroducing cross-attention 
layers into the model architecture, we turn\ndiffusion models into powerful and flexible generators for general 
conditioning\ninputs such as text or bounding boxes and high-resolution synthesis becomes\npossible in a 
convolutional manner. Our latent diffusion models (LDMs) achieve\na new state of the art for image inpainting and 
highly competitive performance\non various tasks, including unconditional image generation, semantic 
scene\nsynthesis, and super-resolution, while significantly reducing computational\nrequirements compared to 
pixel-based DMs. Code is available at\nhttps://github.com/CompVis/latent-diffusion .'
}


Document 10
 - # Chunks: 155
 - Metadata: 


{
    'Published': '2021-02-26',
    'Title': 'Learning Transferable Visual Models From Natural Language Supervision',
    'Authors': 'Alec Radford, Jong Wook Kim, Chris Hallacy, Aditya Ramesh, Gabriel Goh, Sandhini Agarwal, Girish 
Sastry, Amanda Askell, Pamela Mishkin, Jack Clark, Gretchen Krueger, Ilya Sutskever',
    'Summary': 'State-of-the-art computer vision systems are trained to predict a fixed set\nof predetermined 
object categories. This restricted form of supervision limits\ntheir generality and usability since additional 
labeled data is needed to\nspecify any other visual concept. Learning directly from raw text about images\nis a 
promising alternative which leverages a much broader source of\nsupervision. We demonstrate that the simple 
pre-training task of predicting\nwhich caption goes with which image is an efficient and scalable way to 
learn\nSOTA image representations from scratch on a dataset of 400 million (image,\ntext) pairs collected from the 
internet. After pre-training, natural language\nis used to reference learned visual concepts (or describe new ones)
enabling\nzero-shot transfer of the model to downstream tasks. We study the performance\nof this approach by 
benchmarking on over 30 different existing computer vision\ndatasets, spanning tasks such as OCR, action 
recognition in videos,\ngeo-localization, and many types of fine-grained object classification. The\nmodel 
transfers non-trivially to most tasks and is often competitive with a\nfully supervised baseline without the need 
for any dataset specific training.\nFor instance, we match the accuracy of the original ResNet-50 on 
ImageNet\nzero-shot without needing to use any of the 1.28 million training examples it\nwas trained on. We release
our code and pre-trained model weights at\nhttps://github.com/OpenAI/CLIP.'
}

### **任务 2：** 构建文档向量存储

我们现在已经有了所有组件，可以继续围绕它们创建索引：

In [18]:
%%time
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

Constructing Vector Stores
CPU times: user 1.85 s, sys: 115 ms, total: 1.97 s
Wall time: 54.5 s


<br>

接着像下面这样把索引合并为一个：

In [19]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 695 chunks


<br>

### **任务 3：[练习]** 实现 RAG 链

终于，一切准备就绪，来实现 RAG 工作流吧！回顾一下，我们现在有：

* 一种用向量存储从零创建对话记忆的方法（用 `default_FAISS()` 初始化）
* 通过 `ArxivLoader` 预加载了包括文档信息的向量存储（存在 `docstore` 里）。

再借助几个工具，就能集成您的链了！我们还提供了几个额外的便捷工具（`doc2str` 及 `RPrint`），您可以酌情使用。此外，一些启动提示词和结构已经定义好了。

> **基于上述这些：** 实现 `retrieval_chain` 吧。

In [20]:
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr
from functools import partial
from operator import itemgetter

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
# instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

stream_chain = chat_prompt| RPrint() | instruct_llm | StrOutputParser()

################################################################################################
## BEGIN TODO: Implement the retrieval chain to make your system work!

retrieval_chain = (
    {'input' : (lambda x: x)}
    ## TODO: Make sure to retrieve history & context from convstore & docstore, respectively.
    ## HINT: Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
    | RunnableAssign({'history' : lambda d: None})
    | RunnableAssign({'context' : lambda d: None})
)
retrieval_chain = (
    {'input' : (lambda x: x)}
    ## TODO: Make sure to retrieve history & context from convstore & docstore, respectively.
    ## HINT: Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
    | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context' : itemgetter('input') | docstore.as_retriever()  | long_reorder | docs2str})
    | RPrint()
)
## END TODO
################################################################################################

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "Tell me about RAG!"  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

{
    'input': 'Tell me about RAG!',
    'history': '',
    'context': '[Quote from ArchRAG: Attributed Community-based Hierarchical Retrieval-Augmented Generation] 
.\\nRAG method\\nRetrieval method\\nQuestion\\nAttributed community\\nHierarchical index\\nRetrieval 
filtering\\nZero-shot / CoT [27]\\nNone\\nSpecific\\nVanilla RAG\\nVector search\\nSpecific\\nGraphRAG 
[12]\\nTraverse\\nSpecific&abstract\\nLightRAG [17]\\nExtract keywords + Vector search\\nAbstract\\nHippoRAG 
[18]\\nPPR\\nSpecific\\nArchRAG (ours)\\nHierarchical search\\nSpecific&abstract\\nSpecifically, Microsoft first 
proposes a graph-based RAG system\\ncalled GraphRAG [12], which first builds a knowledge graph (KG)\\nby extracting
entities and relationships from the external corpus,\\nthen employs the Leiden method [54] to detect communities 
and\\ngenerates a summary for each community using LLMs\n[Quote from KET-RAG: A Cost-Efficient Multi-Granular 
Indexing Framework for Graph-RAG] .\\n2.3\\nMicrosoft\\u2019s Graph-RAG\\nMicrosoft proposed the Graph-RAG system 
[8], which constructs a\\nknowledge graph index with multi-level communities and employs\\ntailored strategies for 
both local and global search. In this section,\\nwe focus on its indexing and retrieval operations for local 
search,\\nwhich are relevant to our work.\\nAlgorithm 1 outlines the pseudo-code for constructing the graph\\nindex
G = (V\\n\\ud835\\udc52\\u222aV\\n\\ud835\\udc61, E), where V\\n\\ud835\\udc52and V\\n\\ud835\\udc61represent 
entities and\\ntext chunks, respectively. Given a text chunk set T, KG-Index first\\nKET-RAG: A Cost-Efficient 
Multi-Granular Indexing Framework for Graph-RAG\\nConference acronym \\u2019XX, June 03\\u201305, 2018, Woodstock, 
NY\\nAlgorithm 1: KG-Index (T)\\nInput: The text chunk set T.\\nOutput: A TAG index G\n[Quote from 
Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] . We refer to this decoding procedure as 
\\u201cThorough Decoding.\\u201d For longer\\noutput sequences, |Y | can become large, requiring many forward 
passes. For more ef\\ufb01cient decoding,\\nwe can make a further approximation that p\\u03b8(y|x, zi) \\u22480 
where y was not generated during beam\\nsearch from x, zi. This avoids the need to run additional forward passes 
once the candidate set Y has\\nbeen generated. We refer to this decoding procedure as \\u201cFast 
Decoding.\\u201d\\n3\\nExperiments\\nWe experiment with RAG in a wide range of knowledge-intensive tasks. For all 
experiments, we use\\na single Wikipedia dump for our non-parametric knowledge source. Following Lee et al. [31] 
and\\nKarpukhin et al. [26], we use the December 2018 dump. Each Wikipedia article is split into 
disjoint\\n100-word chunks, to make a total of 21M documents\n[Quote from Retrieval-Augmented Generation for 
Knowledge-Intensive NLP Tasks] . In one approach, RAG-Sequence, the model uses the same document\\nto predict each 
target token. The second approach, RAG-Token, can predict each target token based\\non a different document. In the
following, we formally introduce both models and then describe the\\np\\u03b7 and p\\u03b8 components, as well as 
the training and decoding procedure.\\n2.1\\nModels\\nRAG-Sequence Model\\nThe RAG-Sequence model uses the same 
retrieved document to generate\\nthe complete sequence. Technically, it treats the retrieved document as a single 
latent variable that\\nis marginalized to get the seq2seq probability p(y|x) via a top-K approximation\n'
}

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Tell me about RAG!\n\n From this, we have retrieved the following potentially-useful info:  
Conversation History Retrieval:\n\n\n Document Retrieval:\n[Quote from ArchRAG: Attributed Community-based 
Hierarchical Retrieval-Augmented Generation] .\\nRAG method\\nRetrieval method\\nQuestion\\nAttributed 
community\\nHierarchical index\\nRetrieval filtering\\nZero-shot / CoT [27]\\nNone\\nSpecific\\nVanilla 
RAG\\nVector search\\nSpecific\\nGraphRAG [12]\\nTraverse\\nSpecific&abstract\\nLightRAG [17]\\nExtract keywords + 
Vector search\\nAbstract\\nHippoRAG [18]\\nPPR\\nSpecific\\nArchRAG (ours)\\nHierarchical 
search\\nSpecific&abstract\\nSpecifically, Microsoft first proposes a graph-based RAG system\\ncalled GraphRAG 
[12], which first builds a knowledge graph (KG)\\nby extracting entities and relationships from the external 
corpus,\\nthen employs the Leiden method [54] to detect communities and\\ngenerates a summary for each community 
using LLMs\n[Quote from KET-RAG: A Cost-Efficient Multi-Granular Indexing Framework for Graph-RAG] 
.\\n2.3\\nMicrosoft\\u2019s Graph-RAG\\nMicrosoft proposed the Graph-RAG system [8], which constructs a\\nknowledge
graph index with multi-level communities and employs\\ntailored strategies for both local and global search. In 
this section,\\nwe focus on its indexing and retrieval operations for local search,\\nwhich are relevant to our 
work.\\nAlgorithm 1 outlines the pseudo-code for constructing the graph\\nindex G = 
(V\\n\\ud835\\udc52\\u222aV\\n\\ud835\\udc61, E), where V\\n\\ud835\\udc52and V\\n\\ud835\\udc61represent entities 
and\\ntext chunks, respectively. Given a text chunk set T, KG-Index first\\nKET-RAG: A Cost-Efficient 
Multi-Granular Indexing Framework for Graph-RAG\\nConference acronym \\u2019XX, June 03\\u201305, 2018, Woodstock, 
NY\\nAlgorithm 1: KG-Index (T)\\nInput: The text chunk set T.\\nOutput: A TAG index G\n[Quote from 
Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] . We refer to this decoding procedure as 
\\u201cThorough Decoding.\\u201d For longer\\noutput sequences, |Y | can become large, requiring many forward 
passes. For more ef\\ufb01cient decoding,\\nwe can make a further approximation that p\\u03b8(y|x, zi) \\u22480 
where y was not generated during beam\\nsearch from x, zi. This avoids the need to run additional forward passes 
once the candidate set Y has\\nbeen generated. We refer to this decoding procedure as \\u201cFast 
Decoding.\\u201d\\n3\\nExperiments\\nWe experiment with RAG in a wide range of knowledge-intensive tasks. For all 
experiments, we use\\na single Wikipedia dump for our non-parametric knowledge source. Following Lee et al. [31] 
and\\nKarpukhin et al. [26], we use the December 2018 dump. Each Wikipedia article is split into 
disjoint\\n100-word chunks, to make a total of 21M documents\n[Quote from Retrieval-Augmented Generation for 
Knowledge-Intensive NLP Tasks] . In one approach, RAG-Sequence, the model uses the same document\\nto predict each 
target token. The second approach, RAG-Token, can predict each target token based\\non a different document. In the
following, we formally introduce both models and then describe the\\np\\u03b7 and p\\u03b8 components, as well as 
the training and decoding procedure.\\n2.1\\nModels\\nRAG-Sequence Model\\nThe RAG-Sequence model uses the same 
retrieved document to generate\\nthe complete sequence. Technically, it treats the retrieved document as a single 
latent variable that\\nis marginalized to get the seq2seq probability p(y|x) via a top-K approximation\n\n\n 
(Answer only from retrieval. Only cite sources that are used. Make your response conversational.)'
        ),
        HumanMessage(content='Tell me about RAG!')
    ]
)

RAG, or Retrieval-Augmented Generation, is a method used in natural language processing that combines retrieval and generation techniques to improve the performance of language models on knowledge-intensive tasks.

The RAG method was first proposed by Microsoft and is described in the paper "ArchRAG: Attributed Community-based Hierarchical Retrieval-Augmented Generation." According to the paper, RAG involves building a knowledge graph from an external corpus and then using this graph to retrieve relevant information for a given input prompt. The retrieved information is then used to generate a response using large language models (LLMs).

There are different variants of RAG, including vanilla RAG, GraphRAG, LightRAG, and ArchRAG. Vanilla RAG uses a simple vector search method to retrieve relevant information, while GraphRAG uses a graph-based approach to build a knowledge graph and retrieves information using a traversal method. LightRAG extracts keywords and uses vector search to retr

### **任务 4：** 与 Gradio 聊天机器人交互

In [22]:
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

/usr/local/lib/python3.11/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.41.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://9559e568fdd1714603.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9559e568fdd1714603.gradio.live
Closing server running on port: 7860


<br>

----

<br>

## **第 4 部分：** 保存索引以用于评估

实现 RAG 链后，请参考[官方文档](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading)保存您积累出来的向量存储。最后的评估会用到！

In [24]:
## Save and compress your index
docstore.save_local("docstore_index")
!tar czvf docstore_index.tgz docstore_index

!rm -rf docstore_index

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


如果所有内容都已正确保存，就可以执行以下代码从 `tgz` 压缩文件拿到索引了（只要安装好了 pip 环境）。当您确认这个代码单元能拿到您的索引之后，把 `docstore_index.tgz` 下载下来，下个 notebook 会用到！

In [25]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
!tar xzvf docstore_index.tgz
new_db = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = new_db.similarity_search("Testing the index")
print(docs[0].page_content[:1000])

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss
.\n2.3\nMicrosoft\u2019s Graph-RAG\nMicrosoft proposed the Graph-RAG system [8], which constructs a\nknowledge graph index with multi-level communities and employs\ntailored strategies for both local and global search. In this section,\nwe focus on its indexing and retrieval operations for local search,\nwhich are relevant to our work.\nAlgorithm 1 outlines the pseudo-code for constructing the graph\nindex G = (V\n\ud835\udc52\u222aV\n\ud835\udc61, E), where V\n\ud835\udc52and V\n\ud835\udc61represent entities and\ntext chunks, respectively. Given a text chunk set T, KG-Index first\nKET-RAG: A Cost-Efficient Multi-Granular Indexing Framework for Graph-RAG\nConference acronym \u2019XX, June 03\u201305, 2018, Woodstock, NY\nAlgorithm 1: KG-Index (T)\nInput: The text chunk set T.\nOutput: A TAG index G


----

## **第 5 部分：** 总结

恭喜！如果您的 RAG 链能正常运行，就继续进入 08_evaluation.ipynb 进行 **RAG 评估**吧！

### <font color="#76b900">**非常好！**</font>

### **接下来**：
**[可选]** 回顾 notebook 顶部的“思考问题”。